In [28]:
#Libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
!pip install contractions
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=5f0142887ed322feaebdd5dbc83bd75e632399fd56179766eb747c03c4e16a75
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [3]:
#imports
import string # from some string manipulation tasks
import nltk # natural language toolkit
import re # regex
from string import punctuation # solving punctuation problems
from nltk.corpus import stopwords # stop words in sentences
from nltk.stem import WordNetLemmatizer # For stemming the sentence
from nltk.stem import SnowballStemmer # For stemming the sentence
from contractions import contractions_dict # to solve contractions
from autocorrect import Speller #correcting the spellings

In [5]:
data = pd.read_csv('comments.csv')
data.columns
data1=data.drop(['Unnamed: 0','Likes','Time','user','UserLink'],axis=1)
data1

,Comment
0,Play on with a super powerful processor and s...
1,Ok this is awesome. Im so happy that Samsung i...
2,"Ever since the S22 Ultra, I've been the bigges..."
3,By far the most creative and captivating intro...
4,Probably one of the best smartphone trailers E...
...,...
815,Samsung is always amazing
816,But i phone is dream for everyone
817,"Whatever they produce, after 2 years you have ..."
818,I would hate to drop this phone on the floor


In [6]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data1["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data1["Comment"]]
data1["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data1["Comment"]]
data1["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data1["Comment"]]
data1['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data1["Comment"]]
score = data1["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
data1["Sentiment"] = sentiment
data1.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,Comment,Positive,Negative,Neutral,Compound,Sentiment
0,Play on with a super powerful processor and s...,0.349,0.000,0.651,0.8442,Positive
1,Ok this is awesome. Im so happy that Samsung i...,0.358,0.018,0.624,0.9735,Positive
2,"Ever since the S22 Ultra, I've been the bigges...",0.259,0.000,0.741,0.9059,Positive
3,By far the most creative and captivating intro...,0.189,0.000,0.811,0.5400,Positive
4,Probably one of the best smartphone trailers E...,0.321,0.000,0.679,0.7955,Positive


In [7]:
data2=data1.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head()

,Comment,Sentiment
0,Play on with a super powerful processor and s...,Positive
1,Ok this is awesome. Im so happy that Samsung i...,Positive
2,"Ever since the S22 Ultra, I've been the bigges...",Positive
3,By far the most creative and captivating intro...,Positive
4,Probably one of the best smartphone trailers E...,Positive


In [8]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [9]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)
    
    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    #text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    # text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [10]:
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_processing(text))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [11]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [12]:
processed_data = {
    'Sentence':data_copy.Comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,play on with a super powerful processor and sn...,2
1,ok this is awesome im so happy that samsung is...,2
2,ever since the s22 ultra ive been the biggest ...,2
3,by far the most creative and captivating intro...,2
4,probably one of the best smartphone trailer ev...,2


In [13]:
processed_data['Sentence']

0      play on with a super powerful processor and sn...
1      ok this is awesome im so happy that samsung is...
2      ever since the s22 ultra ive been the biggest ...
3      by far the most creative and captivating intro...
4      probably one of the best smartphone trailer ev...
                             ...                        
815                            samsung is always amazing
816                    but i phone is dream for everyone
817    whatever they produce after 2 year you have to...
818         i would hate to drop this phone on the floor
819                             my phone is samsung a10s
Name: Sentence, Length: 820, dtype: object

In [14]:
processed_data.tail()

,Sentence,Sentiment
815,samsung is always amazing,2
816,but i phone is dream for everyone,2
817,whatever they produce after 2 year you have to...,2
818,i would hate to drop this phone on the floor,0
819,my phone is samsung a10s,1


In [15]:
processed_data['Sentiment'].value_counts()

2    532
1    212
0     76
Name: Sentiment, dtype: int64

In [16]:
NegCom = processed_data.Sentence[processed_data["Sentiment"]==0]

In [17]:
neg_comments=[]
for n in NegCom:
   neg_comments.append(n)
pd.DataFrame({"Negative Comments":neg_comments})
   

,Negative Comments
0,hmmm still not convincing me enough to move fr...
1,samsung official introduction are not a good a...
2,the dumb equipment of 700 performance better
3,galaxy m13 is bad at performance
4,it the worst that can exist
...,...
71,dumbest commercial ever my god that wa painful...
72,cool and all but they bragged how fast the 22 ...
73,samsung just stick to one phone for atleast 2 ...
74,boring


In [18]:
df_neutral = processed_data[(processed_data['Sentiment']==1)] 
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_negative_upsampled = resample(df_negative, 
                                 replace=True,    
                                 n_samples= 532, 
                                 random_state=42)  

df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,    
                                 n_samples= 532, 
                                 random_state=42)  


# Concatenate the upsampled dataframes with the neutral dataframe
final_data = pd.concat([df_negative_upsampled,df_neutral_upsampled,df_positive])

In [19]:
final_data['Sentiment'].value_counts()

0    532
1    532
2    532
Name: Sentiment, dtype: int64

In [20]:
final_data.tail()

,Sentence,Sentiment
812,bring s pen in budget like note 10 lite at lea...,2
813,nothing new but nice marketing,2
815,samsung is always amazing,2
816,but i phone is dream for everyone,2
817,whatever they produce after 2 year you have to...,2


##Data Transformation

In [29]:
x = final_data['Sentence']
Y = final_data['Sentiment']

In [31]:
#import tokenizer input
from b2_tokenizer_input import CustomTokenizerExample

In [43]:
! cat b2_tokenizer_input.py

import spacy
nlp = spacy.load('en_core_web_sm')

import string
punct = string.punctuation
# punct

from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS) # list of stopwords

class CustomTokenizerExample():
    def __init__(self):
        pass

    def text_data_cleaning(self,sentence):
        doc = nlp(sentence)                         # spaCy tokenize text & call doc components, in order

        tokens = [] # list of tokens
        for token in doc:
            if token.lemma_ != "-PRON-":
                temp = token.lemma_.lower().strip()
            else:
              temp = token.lower_
            tokens.append(temp)

        cleaned_tokens = []
        for token in tokens:
            if token not in stopwords and token not in punct:
                cleaned_tokens.append(token)
        return cleaned_tokens

##TF IDF

In [32]:
token = CustomTokenizerExample()

In [33]:
token.text_data_cleaning("Those were the best days of my life!")

['good', 'day', 'life']

In [34]:
tfidf = TfidfVectorizer(tokenizer = token.text_data_cleaning)

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=0.2, random_state=0)

In [37]:
from sklearn.svm import LinearSVC
classifier = LinearSVC()

In [38]:
pipeline = Pipeline([('tfidf',tfidf),('clf',classifier)])

In [39]:
pipeline.fit(x_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<bound method CustomTokenizerExample.text_data_cleaning of <b2_tokenizer_input.CustomTokenizerExample object at 0x7fe5737bde10>>)),
                ('clf', LinearSVC())])

In [40]:
Y_pred = pipeline.predict(x_test)

In [41]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [42]:
# confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[106,   0,   0],
       [  0, 101,   3],
       [  1,  18,  91]])

In [43]:
# classification_report
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       106
           1       0.85      0.97      0.91       104
           2       0.97      0.83      0.89       110

    accuracy                           0.93       320
   macro avg       0.94      0.93      0.93       320
weighted avg       0.94      0.93      0.93       320



In [45]:
#Model Serialization

In [46]:
import joblib
joblib.dump(pipeline,'c2_SentimentAnalysis_Model_Pipeline.pkl')

['c2_SentimentAnalysis_Model_Pipeline.pkl']

In [47]:
#Predict Sentiments using Model

In [48]:
new_comment = []
pred_sentiment = []

while True:
  
  # ask for a new comment
  comment = input("Please input a comment - ")

  if comment == 'skip':
    print("See you soon!")
    break
  else:
    prediction = pipeline.predict([comment])

    if prediction == 0:
      result = 'Negative'
      print("Result: This comment is negative\n")
    if prediction == 1:
      result = 'Neutral'
      print("Result: This comment is neutral\n")
    else:
      result = 'Positive'
      print("Result: This comment is positive\n")
  
  new_comment.append(comment)
  pred_sentiment.append(result)

Please input a comment - wonderful phone, i have never seen this before
Result: This comment is neutral

Please input a comment - bad one
Result: This comment is negative

Result: This comment is positive

Please input a comment - amazing
Result: This comment is positive

Please input a comment - skip
See you soon!
